# Simple Chain Tutorial

In [2]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

from langchain_openai import ChatOpenAI, OpenAI

import os
from dotenv import load_dotenv
load_dotenv()

openai_key = os.getenv("OPENAI_API_KEY")

### 1st: Define the main pieces of the chain
- model: `gpt-4, claude3-opus, mistral7B`
- prompt: instructions pre-query

### 2nd: Define the chain
The output of one component is the input of the following one. 
- We use the `|` as the operator to create the sequence

In [3]:
model = ChatOpenAI(model="gpt-3.5-turbo")

prompt = ChatPromptTemplate.from_template("tell me something interesting about {topic}, and further about {beyond}")

chain = prompt | model | StrOutputParser()

In [5]:
res = chain.invoke({"topic": "Brazil", "beyond": "Rio de Janeiro"})
print(res)

- Brazil is the largest country in South America and the fifth largest in the world by both land area and population. It is known for its diverse culture, vibrant music and dance, and beautiful natural landscapes.

- Rio de Janeiro is one of the most visited cities in the Southern Hemisphere and is famous for its stunning beaches, iconic landmarks like Christ the Redeemer and Sugarloaf Mountain, and its annual Carnival celebration, which is one of the largest in the world.

- Rio de Janeiro was the capital of Brazil from 1763 to 1960, when it was moved to the newly built city of Brasília. Despite no longer being the capital, Rio remains one of the country's most important cultural and economic centers.

- The city is surrounded by lush mountains and forests, including the Tijuca National Park, which is home to a variety of wildlife and waterfalls. The city's unique geography, with its mix of urban areas and natural beauty, makes it a popular destination for outdoor enthusiasts and natu

## Deep Dive into __components__

### 1. Prompt
dict -> PromptValue

In [6]:
prompt_value = prompt.invoke({"topic": "World Cup 2014", "beyond": "Brazil"})
prompt_value

ChatPromptValue(messages=[HumanMessage(content='tell me something interesting about World Cup 2014, and further about Brazil')])

### 2. Model
- LLM (which takes a string as input);
- ChatModel (which takes a sequence of messages as input).

The `PromptValue` -> `model` -> `BaseMessage`

In [7]:
# Passing to a chatmodel
message = model.invoke(prompt_value)
message

AIMessage(content='One interesting fact about the 2014 World Cup in Brazil is that it was the most expensive World Cup ever, with an estimated cost of over $11 billion. This included the construction or renovation of 12 stadiums across the country, as well as infrastructure improvements such as transportation and hotels.\n\nAdditionally, Brazil is the only country to have appeared in every single FIFA World Cup tournament since the competition began in 1930. They have won the tournament a record five times, in 1958, 1962, 1970, 1994, and 2002. The country is known for its passionate football fans and the iconic yellow jerseys worn by the national team, which has produced some of the greatest players in the history of the sport, including Pelé, Ronaldo, and Ronaldinho.', response_metadata={'token_usage': {'completion_tokens': 161, 'prompt_tokens': 22, 'total_tokens': 183}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_b28b39ffa8', 'finish_reason': 'stop', 'logprobs': None}, i

In [8]:
for item in message:
    print(item)

('content', 'One interesting fact about the 2014 World Cup in Brazil is that it was the most expensive World Cup ever, with an estimated cost of over $11 billion. This included the construction or renovation of 12 stadiums across the country, as well as infrastructure improvements such as transportation and hotels.\n\nAdditionally, Brazil is the only country to have appeared in every single FIFA World Cup tournament since the competition began in 1930. They have won the tournament a record five times, in 1958, 1962, 1970, 1994, and 2002. The country is known for its passionate football fans and the iconic yellow jerseys worn by the national team, which has produced some of the greatest players in the history of the sport, including Pelé, Ronaldo, and Ronaldinho.')
('additional_kwargs', {})
('response_metadata', {'token_usage': {'completion_tokens': 161, 'prompt_tokens': 22, 'total_tokens': 183}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_b28b39ffa8', 'finish_reason': 'sto

In [9]:
# Passing to an LLM
llm = OpenAI(model="gpt-3.5-turbo-instruct")
string = llm.invoke(prompt_value)
string

'\n\n\nThe 2014 World Cup in Brazil was the most expensive World Cup in history, with an estimated cost of $15 billion. This is more than three times the cost of the previous World Cup in South Africa in 2010. \n\nFurthermore, Brazil is the only country to have won the World Cup five times, and they hold the record for the most goals scored in World Cup history with 229 goals. The Brazilian team is also the only team to have participated in every World Cup since its inception in 1930. \n\nIn addition, the 2014 World Cup was the first time that goal-line technology was used, which helped referees determine if a goal was scored or not. This technology was introduced after a controversial goal was not awarded to England in the 2010 World Cup. \n\nDespite being the host country, Brazil suffered a humiliating 7-1 loss to Germany in the semi-finals, which is the largest margin of defeat for a host nation in World Cup history. This match also saw Germany score 4 goals in just 6 minutes, makin

### 3. Output Parser
`model(BaseMessage)` -> `Output Parser` -> `(string)`

In [10]:
StrOutputParser().invoke(message)

'One interesting fact about the 2014 World Cup in Brazil is that it was the most expensive World Cup ever, with an estimated cost of over $11 billion. This included the construction or renovation of 12 stadiums across the country, as well as infrastructure improvements such as transportation and hotels.\n\nAdditionally, Brazil is the only country to have appeared in every single FIFA World Cup tournament since the competition began in 1930. They have won the tournament a record five times, in 1958, 1962, 1970, 1994, and 2002. The country is known for its passionate football fans and the iconic yellow jerseys worn by the national team, which has produced some of the greatest players in the history of the sport, including Pelé, Ronaldo, and Ronaldinho.'

# Schemas
Learn more about what come `in` and `out` of every langchain runnable by accessing its JSONSchema representation `.schema()`

In [14]:
# input schemas for the chain and prompt are usually the same
chain.input_schema.schema()
prompt.input_schema.schema()

{'title': 'PromptInput',
 'type': 'object',
 'properties': {'beyond': {'title': 'Beyond', 'type': 'string'},
  'topic': {'title': 'Topic', 'type': 'string'}}}

In [15]:
# model schema
model.input_schema.schema()

{'title': 'ChatOpenAIInput',
 'anyOf': [{'type': 'string'},
  {'$ref': '#/definitions/StringPromptValue'},
  {'$ref': '#/definitions/ChatPromptValueConcrete'},
  {'type': 'array',
   'items': {'anyOf': [{'$ref': '#/definitions/AIMessage'},
     {'$ref': '#/definitions/HumanMessage'},
     {'$ref': '#/definitions/ChatMessage'},
     {'$ref': '#/definitions/SystemMessage'},
     {'$ref': '#/definitions/FunctionMessage'},
     {'$ref': '#/definitions/ToolMessage'}]}}],
 'definitions': {'StringPromptValue': {'title': 'StringPromptValue',
   'description': 'String prompt value.',
   'type': 'object',
   'properties': {'text': {'title': 'Text', 'type': 'string'},
    'type': {'title': 'Type',
     'default': 'StringPromptValue',
     'enum': ['StringPromptValue'],
     'type': 'string'}},
   'required': ['text']},
  'AIMessage': {'title': 'AIMessage',
   'description': 'Message from an AI.',
   'type': 'object',
   'properties': {'content': {'title': 'Content',
     'anyOf': [{'type': 'strin

In [16]:
# chain output schema
chain.output_schema.schema()

{'title': 'StrOutputParserOutput', 'type': 'string'}